In [2]:
!pip install -Uq tensorflow_hub  pydub

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-macos 2.15.0 requires keras<2.16,>=2.15.0, but you have keras 3.6.0 which is incompatible.
tensorflow-macos 2.15.0 requires ml-dtypes~=0.2.0, but you have ml-dtypes 0.4.1 which is incompatible.
tensorflow-macos 2.15.0 requires tensorboard<2.16,>=2.15, but you have tensorboard 2.17.1 which is incompatible.


In [3]:
!pip install -Uq tensorflow-io

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-macos 2.15.0 requires keras<2.16,>=2.15.0, but you have keras 3.6.0 which is incompatible.
tensorflow-macos 2.15.0 requires ml-dtypes~=0.2.0, but you have ml-dtypes 0.4.1 which is incompatible.
tensorflow-macos 2.15.0 requires tensorboard<2.16,>=2.15, but you have tensorboard 2.17.1 which is incompatible.


In [5]:
import tensorflow as tf
import tensorflow_hub as hub

# Load the YAMNET model
model = hub.load('https://tfhub.dev/google/yamnet/1')

# Load an audio file
audio, sample_rate = tf.audio.decode_wav(tf.io.read_file('audio/h1.wav'))
audio = tf.squeeze(audio, axis=-1)

# Generate embeddings
scores, embeddings, log_mel_spectrogram = model(audio)

print(f"Audio embedding shape: {embeddings.shape}")

Audio embedding shape: (34, 1024)


In [6]:
log_mel_spectrogram

<tf.Tensor: shape=(1680, 64), dtype=float32, numpy=
array([[-6.9077554, -6.9077554, -6.9077554, ..., -6.9077554, -6.9077554,
        -6.9077554],
       [-6.9077554, -6.9077554, -6.9077554, ..., -6.9077554, -6.9077554,
        -6.9077554],
       [-6.9077554, -6.9077554, -6.9077554, ..., -6.9077554, -6.9077554,
        -6.9077554],
       ...,
       [-6.9077554, -6.9077554, -6.9077554, ..., -6.9077554, -6.9077554,
        -6.9077554],
       [-6.9077554, -6.9077554, -6.9077554, ..., -6.9077554, -6.9077554,
        -6.9077554],
       [-6.9077554, -6.9077554, -6.9077554, ..., -6.9077554, -6.9077554,
        -6.9077554]], dtype=float32)>

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np

def get_vggish_embedding(audio_path):
    # Load the VGGish model
    model = hub.load('https://tfhub.dev/google/vggish/1')

    # Load and preprocess audio file
    audio, sample_rate = tf.audio.decode_wav(tf.io.read_file(audio_path))
    audio = tf.squeeze(audio, axis=-1)
    
    # Ensure audio is the correct length (0.96 seconds)
    audio = audio[:int(0.96 * sample_rate)]
    if len(audio) < int(0.96 * sample_rate):
        audio = tf.pad(audio, [[0, int(0.96 * sample_rate) - len(audio)]])

    # Generate embedding
    embedding = model(audio)
    return embedding.numpy()

# Example usage
audio_path = "audio/h1.wav"
vggish_embedding = get_vggish_embedding(audio_path)
print(f"VGGish embedding shape: {vggish_embedding.shape}")